In [36]:
import tensorflow as tf
import numpy as np
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize

In [37]:
dataset_iris = datasets.load_iris()

In [43]:
X = dataset_iris.data  # we only take the first two features.
Y = dataset_iris.target

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

logreg = linear_model.LogisticRegression(C=1e5)

logreg.fit(X_train, Y_train)

Z = logreg.predict(X_test)

print("Accuracy is {}".format(logreg.score(X_test, Y_test)))


Accuracy is 0.98


In [32]:
iris = np.column_stack([dataset_iris['data'], dataset_iris['target']])
np.random.shuffle(iris)
#print(iris)
target_multiclass = label_binarize(iris[:, -1], classes=[0, 1, 2])

# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 10
display_step = 1

# Network Parameters
n_hidden_1 = 15 # 1st layer number of neurons
n_hidden_2 = 45 # 2nd layer number of neurons
n_input = 4 # target + 3 classes
n_classes = 3 #total classes

# tf Graph input
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# Store layers weight & bias

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


# Create model
def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Construct model
logits = multilayer_perceptron(X)

# Define loss and optimizer

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Initializing the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(150/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = iris[i:i+batch_size, :4], target_multiclass[i:i+batch_size]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
                                                            Y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: iris[:,:-1], Y: target_multiclass}))

Epoch: 0001 cost=8.987014039
Epoch: 0002 cost=4.276708134
Epoch: 0003 cost=1.971493014
Epoch: 0004 cost=0.602953697
Epoch: 0005 cost=0.688832595
Epoch: 0006 cost=0.541428863
Epoch: 0007 cost=0.322068423
Epoch: 0008 cost=0.158340906
Epoch: 0009 cost=0.045088897
Epoch: 0010 cost=0.017360445
Epoch: 0011 cost=0.027025336
Epoch: 0012 cost=0.017371484
Epoch: 0013 cost=0.019775333
Epoch: 0014 cost=0.017218610
Epoch: 0015 cost=0.017109369
Optimization Finished!
Accuracy: 0.926667
